In [11]:
import pandas as pd
import os
import json

In [28]:
datasets = []

for root, dirs, files in os.walk('../data/datasets/ocr/converted'):
    for file in files:
        if file.endswith(".jsonl"):
            input_file = os.path.join(root, file)
            print(input_file)
            with open(input_file) as f:
                lines = f.read().splitlines()
            df_inter = pd.DataFrame(lines)
            df_inter.columns = ['json_element']
            df_inter['json_element'].apply(json.loads)
            df = pd.json_normalize(df_inter['json_element'].apply(json.loads))
            datasets.append(df)

../data/datasets/ocr/converted/icdar-2017.jsonl
../data/datasets/ocr/converted/overproof.jsonl
../data/datasets/ocr/converted/icdar-2019.jsonl
../data/datasets/ocr/converted/impresso-nzz.jsonl


In [29]:
len(datasets), datasets[0].columns

(4,
 Index(['language', 'filename', 'dataset_name', 'File', 'Date', 'Type',
        'NbAlignedChar', 'ocr.line', 'ocr.sentence', 'ocr.region',
        'groundtruth.line', 'groundtruth.sentence', 'groundtruth.region'],
       dtype='object'))

In [32]:
for dataset in datasets:
    print(dataset['dataset_name'].unique())
    print('No. lines:', dataset['ocr.line']. nunique(), len(dataset['ocr.sentence']), 
          'No. sentences:', dataset['ocr.sentence']. nunique(), len(dataset['ocr.sentence']), 
          'No. regions:', dataset['ocr.region']. nunique(), len(dataset['ocr.region']))

SyntaxError: f-string: unmatched '[' (4215641899.py, line 3)

In [100]:
idx = 9

In [101]:
df.iloc[idx]['ocr.line']

'Frankreich. Paris, 7 Pluv. (17 Jan.)'

In [102]:
df.iloc[idx]['ocr.sentence']

'Frankreich.'

In [103]:
df.iloc[idx]['ocr.region']

'Frankreich. Paris, 7 Pluv. (17 Jan.) Das Tribunal hat nach einem zweimaligen Scratin Dupuy (Mitglied des National-Institutö ) zu seinem Candidaten für den Erhaltunge - Senat ernannt. Im gesezgebcnden Rath erhielt noch niemand die absolute Mehrheit; wohl aber erblikt man unter den in Vorschlag gebrachten Namen die Exdi- rektorrn Merlin, Revcilliere und Treilhard, dexen erster sogar zi Stimmen hatte.'

In [104]:
df.columns

Index(['filename', 'ocr.line', 'ocr.sentence', 'ocr.region',
       'groundtruth.line', 'groundtruth.sentence', 'groundtruth.region'],
      dtype='object')

In [115]:
# ht_raw = " ".join(df['ocr.sentence'].to_list())
# print(f"{len(set(ht_raw.lower()))} characters in human transcription")
# print(f"The following characters have not been system-transcribed: \n{set(ht_raw.lower())-set(st_raw.lower())}")
tokens = ht_raw.split()
WORDS = set(tokens)

In [116]:
def eddi(input_text, reference_words=WORDS, ed_threshold=25, max_unk_tokens=3):
    """ Baseline I: Edit distance -based Baseline
    An edit distance-based baseline: Given a list of valid (reference) words,
    this baseline (called eddi) detects words not in the reference list and 
    changes them to the closest one in the reference list.
    :param input_text: the source text
    :param reference_words: a list of valid words (e.g., computed from the target data) 
    :param ed_threshold: the edit distance threshold below from which a word is replaced
    :param max_unk_tokens: the max number of unknown tokens in the transcribed text 
    :return: the new text
    """
    tokens = input_text.split()
    # Unknown transcribed tokens; proceed only if few
    unknowns = [i for i, w in enumerate(tokens) if w not in reference_words]
    if len(unknowns) > max_unk_tokens:
        return " ".join(tokens)

    for ind in unknowns:
        # Replace each uknown token with the ground truth token w/min edit distance 
        word = tokens[ind]
        min_cer, new_word = 100, word
        for ref in reference_words:
            candidate_min_cer = pywer.cer([ref], [word])
            if candidate_min_cer < min_cer:
                min_cer = candidate_min_cer
                if min_cer < ed_threshold:
                    new_word = ref
    tokens[ind] = new_word
    
    return " ".join(tokens)